In [9]:
import sqlite3
import pandas as pd
import numpy as np
import pymysql

In [10]:
connection = sqlite3.connect("database.sqlite")
crsr = connection.cursor()

In [11]:
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [12]:
df = pd.DataFrame()

In [13]:
query_match = '''SELECT * FROM Match'''
query_team = '''SELECT * FROM Team'''
query_country = '''SELECT * FROM Country'''
query_team_attributes = '''SELECT * FROM Team_Attributes'''

In [14]:
match_df = sql_query(query_match)
match_df.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [15]:
team_df = sql_query(query_team)
team_df.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [16]:
country_df = sql_query(query_country)
country_df.head()

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy


In [17]:
team_attributes_df = sql_query(query_team_attributes)
team_attributes_df.head()

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


In [18]:
df['season'] = match_df['season']

In [19]:
df['home_team_name'] = match_df['home_team_api_id'].apply(lambda x: (sql_query(("SELECT team_short_name FROM Team WHERE team_api_id IS " + str(x)))).values[0][0])
df['away_team_name'] = match_df['away_team_api_id'].apply(lambda x: (sql_query(("SELECT team_short_name FROM Team WHERE team_api_id IS " + str(x)))).values[0][0])

In [20]:
df['country'] = match_df['country_id'].apply(lambda x: (sql_query(("SELECT name FROM Country WHERE id IS " + str(x)))).values[0][0])

In [22]:
df['result'] = None
df['result'] = np.where(match_df['home_team_goal'] == match_df['away_team_goal'], 0, df['result'])
df['result'] = np.where(match_df['home_team_goal'] > match_df['away_team_goal'], 1, df['result'])
df['result'] = np.where(match_df['home_team_goal'] < match_df['away_team_goal'], 2, df['result'])

In [23]:
pd.set_option('display.max_columns', None)

In [24]:
df.head()

,season,home_team_name,away_team_name,country,result
0,2008/2009,GEN,BAC,Belgium,0
1,2008/2009,ZUL,LOK,Belgium,0
2,2008/2009,CEB,AND,Belgium,2
3,2008/2009,GEN,MON,Belgium,1
4,2008/2009,DEN,STL,Belgium,2


In [120]:
username = "admin"
password = "maestra1"
host = "database.ce3qv1igpszx.us-east-1.rds.amazonaws.com" 
port = 3306

In [121]:
db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor
)
cursor = db.cursor()

In [128]:
create_db = '''CREATE DATABASE databasefutbol'''
cursor.execute(create_db)

1

In [129]:
cursor.execute('SHOW databases')
cursor.fetchall()

[{'Database': 'databasefutbol'},
 {'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'sys'}]

In [131]:
cursor.connection.commit()
use_db = ''' USE databasefutbol'''
cursor.execute(use_db)

0

In [ ]:
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(user = username, pw = password, host = host, db = 'databasefutbol'))

In [140]:
df.to_sql(name='prediccion', con=engine, if_exists= 'append', index=False)

In [141]:
db.commit()

In [142]:
sql = '''SELECT * FROM prediccion'''
cursor.execute(sql)
mi_tabla = cursor.fetchall()
mi_tabla

[{'season': '2008/2009',
  'home_team_name': 'GEN',
  'away_team_name': 'BAC',
  'country': 'Belgium',
  'result': 0},
 {'season': '2008/2009',
  'home_team_name': 'ZUL',
  'away_team_name': 'LOK',
  'country': 'Belgium',
  'result': 0},
 {'season': '2008/2009',
  'home_team_name': 'CEB',
  'away_team_name': 'AND',
  'country': 'Belgium',
  'result': 2},
 {'season': '2008/2009',
  'home_team_name': 'GEN',
  'away_team_name': 'MON',
  'country': 'Belgium',
  'result': 1},
 {'season': '2008/2009',
  'home_team_name': 'DEN',
  'away_team_name': 'STL',
  'country': 'Belgium',
  'result': 2},
 {'season': '2008/2009',
  'home_team_name': 'MEC',
  'away_team_name': 'CLB',
  'country': 'Belgium',
  'result': 0},
 {'season': '2008/2009',
  'home_team_name': 'ROS',
  'away_team_name': 'KOR',
  'country': 'Belgium',
  'result': 0},
 {'season': '2008/2009',
  'home_team_name': 'TUB',
  'away_team_name': 'MOU',
  'country': 'Belgium',
  'result': 2},
 {'season': '2008/2009',
  'home_team_name': 'WE

In [146]:
db.close()

In [147]:
conn = engine.connect()
conn.close()
conn = engine.dispose()